In [1]:
import os
import pandas as pd
import numpy as np
import json

# print('当前工作目录:', os.getcwd())
ocr_types_dict = {
    'OmniDocBench_demo_text_ocr': 'OmniDocBench_demo_text_ocr'
}

result_folder = '../result'

# match_name = 'quick_match'

In [ ]:
# overall result: not distinguishing between Chinese and English, page-level average

dict_list = []

for ocr_type in ocr_types_dict.values():
    # result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    result_path = os.path.join(result_folder, f'{ocr_type}_metric_result.json')

    with open(result_path, 'r') as f:
        result = json.load(f)
    
    save_dict = {}

    for category_type, metric in [("text_block", "Edit_dist"), ("display_formula", "CDM"), ("table", "TEDS"), ("table", "TEDS_structure_only"), ("reading_order", "Edit_dist")]:
        if metric == 'CDM' or metric == "TEDS" or metric == "TEDS_structure_only":
            if result[category_type]["page"].get(metric):
                save_dict[category_type+'_'+metric] = result[category_type]["page"][metric]["ALL"] * 100   # page级别的avg
            else:
                save_dict[category_type+'_'+metric] = 0
        else:
            save_dict[category_type+'_'+metric] = result[category_type]["all"][metric].get("ALL_page_avg", np.nan)

    dict_list.append(save_dict)
    
df = pd.DataFrame(dict_list, index=ocr_types_dict.keys()).round(3)
df['overall'] = ((1-df['text_block_Edit_dist'])*100 + df['display_formula_CDM'] + df['table_TEDS'])/3
# df.to_csv('./overall.csv')

df

In [ ]:
# PDF type
dict_list = []

for ocr_type in ocr_types_dict.values():
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['text_block']["page"]["Edit_dist"])
df2 = pd.DataFrame(dict_list, index=ocr_types_dict.keys())

reordered_df2 = df2.round(3)

selected_columns = reordered_df2[["data_source: book", "data_source: PPT2PDF", "data_source: research_report", "data_source: colorful_textbook", "data_source: exam_paper", "data_source: magazine", "data_source: academic_literature", "data_source: note", "data_source: newspaper"]]
# calculate mean
selected_columns['mean'] = reordered_df2["ALL"]
# selected_columns['variance'] = selected_columns.var(axis=1)
# selected_columns.to_csv('./data_source.csv')

selected_columns

In [ ]:
# page special issue
dict_list = []

for ocr_type in ocr_types_dict.values():
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['text_block']["page"]["Edit_dist"])
df2 = pd.DataFrame(dict_list, index=ocr_types_dict.keys())

reordered_df2 = df2.round(3)
reordered_df2

# selected_columns = reordered_df2[['fuzzy_scan', 'watermark', 'colorful_backgroud']] # use this for full dataset
selected_columns = reordered_df2[[col for col in ['fuzzy_scan', 'watermark', 'colorful_backgroud'] if col in reordered_df2.columns]]

# selected_columns.to_csv('./page_issue.csv')
selected_columns

In [ ]:
# reading order under different layout
dict_list = []

for ocr_type in ocr_types_dict.values():
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['reading_order']["page"]["Edit_dist"])

df3 = pd.DataFrame(dict_list, index=ocr_types_dict.keys())

reordered_df3 = df3.round(3)


selected_columns3 = reordered_df3[["layout: single_column", "layout: double_column", "layout: three_column", "layout: other_layout"]]

# selected_columns3.to_csv('./layout.csv')
selected_columns3

In [ ]:
# text attribute
dict_list = []

for ocr_type in ocr_types_dict.values():
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['text_block']["group"]["Edit_dist"])

df4 = pd.DataFrame(dict_list, index=ocr_types_dict.keys())
df4 = df4.round(3)

selected_columns = df4[["text_language: text_english", "text_language: text_simplified_chinese", "text_language: text_en_ch_mixed", "text_background: white", "text_background: single_colored", "text_background: multi_colored"]]

# selected_columns.to_csv('.text_attribute.csv')
selected_columns

In [ ]:
# table attribute
dict_list = []

for ocr_type in ocr_types_dict.values():
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['table']["group"]["TEDS"])

df4 = pd.DataFrame(dict_list, index=ocr_types_dict.keys())
df4 = df4 * 100
df4 = df4.round(1)

selected_columns = df4[[col for col in [
    "language: table_en", 
    "language: table_simplified_chinese", 
    "language: table_en_ch_mixed", 
    "line: full_line", 
    "line: less_line", 
    "line: fewer_line", 
    "line: wireless_line", 
    "with_span: True", 
    "with_span: False", 
    "include_equation: True", 
    "include_equation: False", 
    "include_background: True", 
    "include_background: False", 
    "table_layout: vertical", 
    "table_layout: horizontal"
] if col in df4.columns]]

# selected_columns.to_csv('./table_attribute.csv')
selected_columns

In [17]:
# text Recognition

# ocr_types = ['OmniDocBench_easyocr_text_ocr', 'OmniDocBench_openocr_text_ocr']
ocr_types = ['OmniDocBench_demo_text_ocr']
offical_names = ocr_types

result_folder = '../result'

dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result["group"]["Edit_dist"])

df4 = pd.DataFrame(dict_list, index=offical_names)
df4 = df4.round(3)

selected_columns = df4[[col for col in [
    "text_language: text_english", 
    "text_language: text_simplified_chinese", 
    "text_language: text_en_ch_mixed", 
    "text_background: white", 
    "text_background: single_colored", 
    "text_background: multi_colored", 
    "text_rotate: normal", 
    "text_rotate: rotate90", 
    "text_rotate: rotate270", 
    "text_rotate: horizontal"
] if col in df4.columns]]


selected_columns.to_csv('.text_attribute.csv')
selected_columns

,text_language: text_english,text_language: text_simplified_chinese,text_language: text_en_ch_mixed,text_background: white,text_background: single_colored,text_background: multi_colored,text_rotate: normal,text_rotate: horizontal
OmniDocBench_demo_text_ocr,0.066,0.053,0.077,0.054,0.136,0.035,0.058,0.0


In [ ]:
# table Recognition

ocr_types = ['OmniDocBench_rapidtable_ocr']
offical_names = ocr_types

result_folder = '../result'

dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['table']["group"]["TEDS"])

df4 = pd.DataFrame(dict_list, index=offical_names)
df4 = df4 * 100
df4 = df4.round(1)

selected_columns = df4[[col for col in [
    "language: table_en", 
    "language: table_simplified_chinese", 
    "language: table_en_ch_mixed", 
    "line: full_line", 
    "line: less_line", 
    "line: fewer_line", 
    "line: wireless_line", 
    "with_span: True", 
    "with_span: False", 
    "include_equation: True", 
    "include_equation: False", 
    "include_background: True", 
    "include_background: False", 
    "table_layout: vertical", 
    "table_layout: horizontal"
] if col in df4.columns]]

# selected_columns.to_csv('./table_attribute.csv')
selected_columns